# RAG

A continuación se muestra una descripción general de alto nivel del sistema que queremos construir:


<img src='images/img_1.png' width="800">

# PARTE I

Empecemos cargando las variables de entorno que necesitamos utilizar.

## Setting up the model
Definamos el modelo LLM que utilizaremos como parte del flujo de trabajo.

In [3]:
import os
from dotenv import load_dotenv


load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Este es el video de YouTube que vamos a utilizar.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=F8NKVhkZZWI&t=1s"

In [8]:
import google.generativeai as genai

# Configurar el modelo de Gemini
model = genai.GenerativeModel('gemini-1.5-pro')  # Usa el modelo que prefieras

Probamos el modelo haciendo una pregunta sencilla

In [9]:
# Probar el modelo con una pregunta sencilla
pregunta_sencilla = "¿Cuál es la capital de Francia?"
respuesta = model.generate_content(pregunta_sencilla)

# Imprimir la respuesta
print(respuesta.text)

La capital de Francia es París.



El resultado del modelo es una instancia de `AIMessage` que contiene la respuesta. Podemos extraer esta respuesta encadenando el modelo con un analizador de salida [outputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Así es como se ve el encadenamiento del modelo con un analizador de salida:

<img src='images/chain1.png' width="1200">

Para este ejemplo, utilizaremos un `StrOutputParser` simple para extraer la respuesta como una cadena.

In [10]:
from langchain_core.output_parsers import StrOutputParser

def gemini_invoke(input_text):
    # Verificar si input_text es un diccionario de LangChain
    if isinstance(input_text, dict) and "messages" in input_text:
        # Extraer el contenido del mensaje del usuario
        messages = input_text["messages"]
        if messages and hasattr(messages[0], "content"):
            input_text = messages[0].content

    # Asegurar que el input sea string
    if not isinstance(input_text, str):
        input_text = str(input_text)

    response = model.generate_content(input_text)  
    return response.text


# Creamos un analizador de salida (StrOutputParser) para asegurarnos de que la respuesta sea una cadena
parser = StrOutputParser()

# Creamos la cadena combinando el modelo y el parser
chain = gemini_invoke | parser

# Probamos la cadena con una pregunta sencilla
pregunta_sencilla = "¿Cuál es la capital de Alemania?"
respuesta_parseada = chain.invoke(pregunta_sencilla)

# Imprimimos la respuesta (ahora debería ser una cadena directamente)
print("Respuesta:", respuesta_parseada)
print("Tipo de respuesta:", type(respuesta_parseada))  # Para verificar que es un string

Respuesta: La capital de Alemania es Berlín.

Tipo de respuesta: <class 'str'>


## Presentamos las plantillas de preguntas

Queremos contextualizar el modelo y la pregunta. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) Son una forma sencilla de definir y reutilizar indicaciones.

In [11]:
from langchain.prompts import ChatPromptTemplate

# Definir la plantilla de pregunta
template = """
Responda la pregunta según el contexto descrito a continuación. Si no puede responder, responda "No lo sé".

Contexto: {contexto}

Pregunta: {pregunta}
"""

# Crear la plantilla de prompt
prompt = ChatPromptTemplate.from_template(template)

# Probar la plantilla con un ejemplo
contexto_ejemplo = "París es la capital de Francia y una de las ciudades más visitadas del mundo."
pregunta_ejemplo = "¿Cuál es la capital de Francia?"

# Formatear el prompt con los valores de contexto y pregunta
formatted_prompt = prompt.format(contexto=contexto_ejemplo, pregunta=pregunta_ejemplo)

# Usar el modelo de Gemini para generar una respuesta
respuesta = model.generate_content(formatted_prompt)

# Imprimir la respuesta
print("Respuesta:", respuesta.text)

Respuesta: París



Ahora podemos encadenar el mensaje con el modelo y el analizador de salida.

<img src='images/chain2.png' width="1200">

In [12]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

chain = (
    RunnableParallel(
        contexto=RunnablePassthrough(),
        pregunta=RunnablePassthrough()
    )
    | prompt
    | RunnableLambda(gemini_invoke)
    | parser
)


## Combinación de cadenas

Podemos combinar diferentes cadenas para crear flujos de trabajo más complejos. Por ejemplo, creemos una segunda cadena que traduzca la respuesta de la primera a otro idioma.

Comencemos creando una nueva plantilla de solicitud para la cadena de traducción:

In [13]:
translation_prompt = ChatPromptTemplate.from_template(
    "Traduce {answer} al {language}"
)


Ahora podemos crear una nueva cadena de traducción que combine el resultado de la primera cadena con la solicitud de traducción.

Así es como se ve el nuevo flujo de trabajo:

<img src='images/chain3.png' width="1200">

In [22]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

# Plantilla para asegurar respuestas directas y concisas
qa_template = ChatPromptTemplate.from_template("""
Responde únicamente con la respuesta correcta basada en el contexto.

Contexto: {contexto}

Pregunta: {pregunta}

Devuelve solo la respuesta sin explicaciones ni comentarios adicionales.
""")

# Función de invocación con limpieza
def clean_gemini_invoke(inputs):
    response = gemini_invoke(inputs)  # Llamamos a Gemini
    if isinstance(response, dict) and "text" in response:
        return response["text"].strip()
    return response.strip()

# Cadena de QA
qa_chain = (
    {
        "contexto": RunnablePassthrough(),
        "pregunta": RunnablePassthrough()
    }
    | qa_template
    | RunnableLambda(clean_gemini_invoke)
)

# Plantilla de traducción simplificada
translation_template = ChatPromptTemplate.from_template("""
Traduce al {language} el siguiente texto sin modificarlo:

{text}
Solo devuelve el texto traducido, sin explicaciones ni estructura JSON.
""")

# Cadena de traducción asegurando solo texto
translation_chain = (
    {
        "text": RunnablePassthrough(),
        "language": lambda _: "Castellano"
    }
    | translation_template
    | RunnableLambda(clean_gemini_invoke)
)

# Función que encadena QA y traducción
def qa_then_translate(inputs):
    respuesta_qa = qa_chain.invoke(inputs)  # Obtenemos respuesta en inglés
    return translation_chain.invoke({"text": respuesta_qa})  # Traducimos solo el texto

# Cadena combinada con correcciones
combined_chain = RunnableLambda(qa_then_translate)

# Prueba final
inputs = {
    "contexto": "París es la capital de Francia y una de las ciudades más visitadas del mundo.",
    "pregunta": "¿Cuál es la capital de Francia?"
}

respuesta_traducida = combined_chain.invoke(inputs)

print(respuesta_traducida)  # Eliminamos "Respuesta traducida:"





París


# PARTE II

## Transcripcion de video de YouTube

El contexto que queremos enviar al modelo proviene de un video de YouTube. Descargamos el video y transcribámoslo con [OpenAI's Whisper](https://openai.com/research/whisper).

Vamos a leer la transcripción y mostrar los primeros caracteres para asegurarnos de que todo funciona como se espera.

## Usando la transcripción completa como contexto

Si intentamos invocar la cadena usando la transcripción como contexto, el modelo devolverá un error porque el contexto es demasiado largo.

Los modelos de lenguaje grandes admiten tamaños de contexto limitados. El vídeo que estamos usando es demasiado largo para que el modelo lo pueda procesar, por lo que necesitamos buscar una solución diferente.

## División de la transcripción

Dado que no podemos usar la transcripción completa como contexto para el modelo, una posible solución es dividir la transcripción en fragmentos más pequeños. Así, podemos invocar el modelo utilizando solo los fragmentos relevantes para responder a una pregunta específica:

<img src='images/system2.png' width="1200">

Comencemos cargando la transcripción en la memoria:

Hay muchas maneras de dividir un documento. En este ejemplo, usaremos un divisor simple que divide el documento en fragmentos de tamaño fijo. Consulta [Divisores de texto](https://python.langchain.com/docs/modules/data_connection/document_transformers/) para obtener más información sobre los diferentes enfoques para dividir documentos.

A modo de ejemplo, dividiremos la transcripción en fragmentos de 100 caracteres con una superposición de 20 caracteres y mostraremos los primeros fragmentos:

Para nuestra aplicación específica, utilizaremos 1000 caracteres en su lugar:

# PARTE III

## Configuración de un Vector Store

Necesitamos una forma eficiente de almacenar fragmentos de documentos, sus Embeddings y realizar búsquedas de similitud a gran escala. Para ello, usaremos un Vector Store.

Un Vector Store es una base de datos de Embeddings especializada en búsquedas rápidas de similitud.


<img src='images/chain4.png' width="1200">

Necesitamos configurar un retriever (https://python.langchain.com/docs/how_to/#retrievers). Este retriever realizará una búsqueda de similitud en el almacén vectorial y devolverá los documentos más similares al siguiente paso de la cadena.

## Configurar Pinecone

Para este ejemplo, usaremos [Pinecone](https://www.pinecone.io/).

<img src="images/pinecone.png" width="800">

El primer paso es crear una cuenta de Pinecone, configurar un índice, obtener una clave API y configurarla como variable de entorno `PINECONE_API_KEY`.

Ahora ejecutemos una búsqueda de similitud en pinecone para asegurarnos de que todo funciona:

Configuremos la nueva cadena usando Pinecone como almacén vectorial: